# Solutions to the Assessment Problems for Computer Infrastructure Module

In [ ]:
#import pandas
import pandas as pd
#import numpy
import numpy as np

## Problem 1: Data from yfinance

In [ ]:
#import yfinance package
import yfinance as yf

## Problem 2: Plotting Data

## Problem 3: Script

## Problem 4: Automation

## End